# Content Based Recommendation System

Score através de cosine similarity

In [0]:
#importando bibliotecas
import numpy as np
import pandas as pd
import os

#Import cosine_similarity function
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
path_market = os.path.join('/content/gdrive/My Drive/Data_Science/CodeNation/projeto_final/data', 'estaticos_market.csv')
path_portfolio1 = os.path.join('/content/gdrive/My Drive/Data_Science/CodeNation/projeto_final/data', 'estaticos_portfolio1.csv')
path_portfolio2 = os.path.join('/content/gdrive/My Drive/Data_Science/CodeNation/projeto_final/data', 'estaticos_portfolio2.csv')
path_portfolio3 = os.path.join('/content/gdrive/My Drive/Data_Science/CodeNation/projeto_final/data', 'estaticos_portfolio3.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
pd.set_option('display.max_rows', 100)
random_seed = 7

In [0]:
df_market = pd.read_csv(path_market, index_col='Unnamed: 0')
df_portfolio1 = pd.read_csv(path_portfolio1, usecols=['id'])
df_portfolio2 = pd.read_csv(path_portfolio2, index_col='Unnamed: 0')
df_portfolio3 = pd.read_csv(path_portfolio3, index_col='Unnamed: 0')

In [0]:
df_market.head()

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,vl_total_tancagem,vl_total_veiculos_antt,vl_total_veiculos_leves,vl_total_veiculos_pesados,fl_optante_simples,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,vl_total_tancagem_grupo,vl_total_veiculos_antt_grupo,vl_potenc_cons_oleo_gas,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,nu_meses_rescencia,...,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_ate_18,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_analfabeto,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,grau_instrucao_macro_desconhecido,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,14.457534,10 a 15,False,False,False,False,False,2005-03-25,False,True,True,SIM,CONSTRUCAO DE EDIFICIOS,CONSTRUCAO,False,False,False,NaN,NaN,NaN,NaN,True,8.0,0.0,0.0,NaN,NaN,NaN,False,RN,VERDE,ACIMA DE 1 ANO,23.0,...,3132172.8,3132172.8,119.0,26.0,100.0,0.0,NaN,1.0,4.0,6.0,5.0,3.0,4.0,2.0,NaN,1.0,NaN,1.0,25.0,NaN,NaN,26.0,19.166667,0.0,0.0,119.0,145.0,13.309195,93.266667,-0.933333,43.738462,93.266667,19.166667,26.0,26.0,27.0,0.0,-3.703704,0.0,0
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,SERVICOS DE ALOJAMENTO/ALIMENTACAO,SERVIÇO,1.463014,1 a 5,False,False,False,True,False,2017-05-12,False,True,True,SIM,ALIMENTACAO,ALOJAMENTO E ALIMENTACAO,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,PI,CINZA,ATE 1 ANO,9.0,...,210000.0,210000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,"TRANSPORTE, ARMAZENAGEM E CORREIO",SERVIÇO,7.093151,5 a 10,False,False,False,True,False,2011-09-26,False,False,True,SIM,TRANSPORTE TERRESTRE,TRANSPORTE ARMAZENAGEM E CORREIO,False,False,False,NaN,NaN,NaN,NaN,True,NaN,0.0,0.0,NaN,NaN,NaN,True,AM,AMARELO,ACIMA DE 1 ANO,26.0,...,50000.0,50000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,SERVICOS DIVERSOS,SERVIÇO,6.512329,5 a 10,False,False,False,False,False,2012-04-25,False,True,True,SIM,REPARACAO E MANUTENCAO DE EQUIPAMENTOS DE INFO...,OUTRAS ATIVIDADES DE SERVICOS,False,False,False,NaN,NaN,NaN,NaN,True,NaN,0.0,0.0,NaN,NaN,NaN,False,AM,AMARELO,ACIMA DE 1 ANO,26.0,...,210000.0,210000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,True,EMPRESARIO INDIVIDUAL,RN,OUTROS,"SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS",SERVIÇO,3.200000,1 a 5,False,False,False,False,False,2015-08-17,False,True,True,SIM,SERVICOS DE ARQUITETURA E ENGENHARIA TESTES E ...,ATIVIDADES PROFISSIONAIS CIENTIFICAS E TECNICAS,False,False,Fals

In [0]:
#Porcentagem de valores por coluna
null_count = df_market.isnull().sum().sort_values(ascending=False)
percent = null_count/df_market.shape[0]
pd.concat([null_count, percent], axis=1, keys=['Total', 'Percent'])

,Total,Percent
qt_alteracao_socio_180d,462298,1.000000
grau_instrucao_macro_desconhecido,462298,1.000000
qt_alteracao_socio_90d,462298,1.000000
qt_alteracao_socio_365d,462298,1.000000
qt_alteracao_socio_total,462298,1.000000
coligada_mais_nova_baixada,462162,0.999706
coligada_mais_antiga_baixada,462162,0.999706
idade_media_coligadas_baixadas,462162,0.999706
vl_total_veiculos_antt,462122,0.999619
vl_total_tancagem,462018,0.999394


In [0]:
#Retirando a categoria 'OUTROS' da variável 'de_ramo'
df_market = df_market.drop(df_market[df_market['de_ramo'] == 'OUTROS'].index)

In [0]:
df_market = df_market.dropna(axis=1)

In [0]:
df_market.columns

Index(['id', 'fl_matriz', 'de_natureza_juridica', 'sg_uf',
       'natureza_juridica_macro', 'de_ramo', 'setor', 'idade_empresa_anos',
       'idade_emp_cat', 'fl_me', 'fl_sa', 'fl_epp', 'fl_mei', 'fl_ltda',
       'fl_st_especial', 'fl_email', 'fl_telefone', 'fl_rm', 'nm_divisao',
       'nm_segmento', 'fl_spa', 'fl_antt', 'fl_veiculo',
       'vl_total_veiculos_pesados_grupo', 'vl_total_veiculos_leves_grupo',
       'fl_simples_irregular', 'fl_passivel_iss', 'qt_filiais'],
      dtype='object')

In [0]:
#Dropando variáveis (abordagem macro)
#cols_to_drop = ['de_natureza_juridica', 'setor', 'idade_emp_cat', 'fl_epp', 'nm_divisao', 'fl_email', 'fl_telefone']

#Dropando variáveis (abordagem micro)
cols_to_drop = ['natureza_juridica_macro', 'de_ramo', 'idade_emp_cat', 'fl_epp', 'nm_segmento', 'fl_email', 'fl_telefone']

df_market = df_market.drop(cols_to_drop, axis=1)
df_market.columns

Index(['id', 'fl_matriz', 'de_natureza_juridica', 'sg_uf', 'setor',
       'idade_empresa_anos', 'fl_me', 'fl_sa', 'fl_mei', 'fl_ltda',
       'fl_st_especial', 'fl_rm', 'nm_divisao', 'fl_spa', 'fl_antt',
       'fl_veiculo', 'vl_total_veiculos_pesados_grupo',
       'vl_total_veiculos_leves_grupo', 'fl_simples_irregular',
       'fl_passivel_iss', 'qt_filiais'],
      dtype='object')

In [0]:
#ID
id_market = df_market['id'].values
df_market = df_market.drop('id', axis=1)

# One hot encoding 
df_market = pd.get_dummies(df_market)

print(df_market.columns)

Index(['fl_matriz', 'idade_empresa_anos', 'fl_me', 'fl_sa', 'fl_mei',
       'fl_ltda', 'fl_st_especial', 'vl_total_veiculos_pesados_grupo',
       'vl_total_veiculos_leves_grupo', 'qt_filiais',
       ...
       'fl_spa_False', 'fl_spa_True', 'fl_antt_False', 'fl_antt_True',
       'fl_veiculo_False', 'fl_veiculo_True', 'fl_simples_irregular_False',
       'fl_simples_irregular_True', 'fl_passivel_iss_False',
       'fl_passivel_iss_True'],
      dtype='object', length=186)


In [0]:
df_market['id'] = id_market

In [0]:
# Escolhendo uma amostra aleatória do portfolio1
df_sample = df_market[df_market['id'].isin(df_portfolio1['id'].sample(frac = 0.8, random_state=random_seed).values)]

#Dropando de novo a variável 'id' de df_market e df_sample
df_market = df_market.drop('id', axis=1)
df_sample = df_sample.drop('id', axis=1)

In [0]:
cosine_sim = cosine_similarity(df_market, df_sample)
cosine_sim

array([[0.27449197, 0.80308226, 0.9409685 , ..., 0.49648732, 0.37522742,
        0.81286054],
       [0.67510067, 0.46727417, 0.60085754, ..., 0.65090513, 0.69629371,
        0.7465898 ],
       [0.41651215, 0.78656542, 0.94087641, ..., 0.60871581, 0.52398469,
        0.8705274 ],
       ...,
       [0.24871812, 0.80938409, 0.94452725, ..., 0.4883403 , 0.35885069,
        0.8000258 ],
       [0.75889306, 0.64550891, 0.6974884 , ..., 0.82541746, 0.71561758,
        0.77231419],
       [0.67980502, 0.37999465, 0.43149076, ..., 0.68389928, 0.76429697,
        0.54188821]])

In [0]:
cosine_sim = np.amax(cosine_sim, axis=1)

In [0]:
df_market['id'] = id_market
df_market['score'] = cosine_sim
df_market.head()

,fl_matriz,idade_empresa_anos,fl_me,fl_sa,fl_mei,fl_ltda,fl_st_especial,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,qt_filiais,de_natureza_juridica_ASSOCIACAO PRIVADA,de_natureza_juridica_AUTARQUIA ESTADUAL OU DO DISTRITO FEDERAL,de_natureza_juridica_AUTARQUIA FEDERAL,de_natureza_juridica_AUTARQUIA MUNICIPAL,de_natureza_juridica_CANDIDATO A CARGO POLITICO ELETIVO,de_natureza_juridica_CLUBE FUNDO DE INVESTIMENTO,de_natureza_juridica_COMUNIDADE INDIGENA,de_natureza_juridica_CONDOMINIO EDILICIO,de_natureza_juridica_CONSORCIO DE EMPREGADORES,de_natureza_juridica_CONSORCIO DE SOCIEDADES,de_natureza_juridica_CONSORCIO PUBDE DIREITO PUB ASS PUB,de_natureza_juridica_CONSORCIO PUBLICO DE DIREITO PRIVADO,de_natureza_juridica_COOPERATIVA,de_natureza_juridica_EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITADA DE NATUREZA EMPRESARIA,de_natureza_juridica_EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITADA DE NATUREZA SIMPLES,de_natureza_juridica_EMPRESA INDIVIDUAL IMOBILIARIA,de_natureza_juridica_EMPRESA PUBLICA,de_natureza_juridica_EMPRESARIO INDIVIDUAL,de_natureza_juridica_ENTIDADE DE MEDIACAO E ARBITRAGEM,de_natureza_juridica_ENTIDADE SINDICAL,de_natureza_juridica_ESTABELECIMENTO NO BRASIL DE FUNDACAO OU ASSOCIACAO ESTRANGEIRAS,de_natureza_juridica_ESTABELECIMENTO NO BRASIL DE SOCIEDADE ESTRANGEIRA,de_natureza_juridica_ESTADO OU DISTRITO FEDERAL,de_natureza_juridica_FRENTE PLEBISCITARIA OU REFERENDARIA,de_natureza_juridica_FUNDACAO PRIVADA,de_natureza_juridica_FUNDACAO PUB DE DIREITO PUB EST OU DO DF,de_natureza_juridica_FUNDACAO PUBLICA DE DIREITO PRIVADO FEFERAL,de_natureza_juridica_FUNDACAO PUBLICA DE DIREITO PRIVADO MUNICIPAL,de_natureza_juridica_FUNDACAO PUBLICA DE DIREITO PUB MUNICIPAL,de_natureza_juridica_FUNDACAO PUBLICA DE DIREITO PUBFEDERAL,...,nm_divisao_FABRICACAO DE PRODUTOS QUIMICOS,nm_divisao_FABRICACAO DE PRODUTOS TEXTEIS,nm_divisao_FABRICACAO DE VEICULOS AUTOMOTORES REBOQUES E CARROCERIAS,nm_divisao_IMPRESSAO E REPRODUCAO DE GRAVACOES,nm_divisao_MANUTENCAO REPARACAO E INSTALACAO DE MAQUINAS E EQUIPAMENTOS,nm_divisao_METALURGIA,nm_divisao_OBRAS DE INFRA ESTRUTURA,nm_divisao_ORGANISMOS INTERNACIONAIS E OUTRAS INSTITUICOES EXTRATERRITORIAIS,nm_divisao_OUTRAS ATIVIDADES DE SERVICOS PESSOAIS,nm_divisao_OUTRAS ATIVIDADES PROFISSIONAIS CIENTIFICAS E TECNICAS,nm_divisao_PESCA E AQUICULTURA,nm_divisao_PESQUISA E DESENVOLVIMENTO CIENTIFICO,nm_divisao_PREPARACAO DE COUROS E FABRICACAO DE ARTEFATOS DE COURO ARTIGOS PARA VIAGEM E CALCADOS,nm_divisao_PRODUCAO FLORESTAL,nm_divisao_PUBLICIDADE E PESQUISA DE MERCADO,nm_divisao_REPARACAO E MANUTENCAO DE EQUIPAMENTOS DE INFORMATICA E COMUNICACAO E DE OBJETOS PESSOAIS E DOMESTICOS,nm_divisao_SEGUROS RESSEGUROS PREVIDENCIA COMPLEMENTAR E PLANOS DE SAUDE,nm_divisao_SELECAO AGENCIAMENTO E LOCACAO DE MAO DE OBRA,nm_divisao_SERVICOS DE ARQUITETURA E ENGENHARIA TESTES E ANALISES TECNICAS,nm_divisao_SERVICOS DE ASSISTENCIA SOCIAL SEM ALOJAMENTO,nm_divisao_SERVICOS DE ESCRITORIO DE APOIO ADMINISTRATIVO E OUTROS SERVICOS PRESTADOS PRINCIPALMENTE AS EMPRESAS,nm_divisao_SERVICOS DOMESTICOS,nm_divisao_SERVICOS ESPECIALIZADOS PARA CONSTRUCAO,nm_divisao_SERVICOS PARA EDIFICIOS E ATIVIDADES PAISAGISTICAS,nm_divisao_TELECOMUNICACOES,nm_divisao_TRANSPORTE AEREO,nm_divisao_TRANSPORTE AQUAVIARIO,nm_divisao_TRANSPORTE TERRESTRE,fl_spa_False,fl_spa_True,fl_antt_False,fl_antt_True,fl_veiculo_False,fl_veiculo_True,fl_simples_irregular_False,fl_simples_irregular_True,fl_passivel_iss_False,fl_passivel_iss_True,id,score
0,True,14.457534,False,False,False,False,False,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,1,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,0.988111
1,True,1.463014,False,False,True,False,False,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb47

In [0]:
df_market = df_market.sort_values('score', ascending=False)
df_market[df_market['score'] > 0.9999999]

,fl_matriz,idade_empresa_anos,fl_me,fl_sa,fl_mei,fl_ltda,fl_st_especial,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,qt_filiais,de_natureza_juridica_ASSOCIACAO PRIVADA,de_natureza_juridica_AUTARQUIA ESTADUAL OU DO DISTRITO FEDERAL,de_natureza_juridica_AUTARQUIA FEDERAL,de_natureza_juridica_AUTARQUIA MUNICIPAL,de_natureza_juridica_CANDIDATO A CARGO POLITICO ELETIVO,de_natureza_juridica_CLUBE FUNDO DE INVESTIMENTO,de_natureza_juridica_COMUNIDADE INDIGENA,de_natureza_juridica_CONDOMINIO EDILICIO,de_natureza_juridica_CONSORCIO DE EMPREGADORES,de_natureza_juridica_CONSORCIO DE SOCIEDADES,de_natureza_juridica_CONSORCIO PUBDE DIREITO PUB ASS PUB,de_natureza_juridica_CONSORCIO PUBLICO DE DIREITO PRIVADO,de_natureza_juridica_COOPERATIVA,de_natureza_juridica_EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITADA DE NATUREZA EMPRESARIA,de_natureza_juridica_EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITADA DE NATUREZA SIMPLES,de_natureza_juridica_EMPRESA INDIVIDUAL IMOBILIARIA,de_natureza_juridica_EMPRESA PUBLICA,de_natureza_juridica_EMPRESARIO INDIVIDUAL,de_natureza_juridica_ENTIDADE DE MEDIACAO E ARBITRAGEM,de_natureza_juridica_ENTIDADE SINDICAL,de_natureza_juridica_ESTABELECIMENTO NO BRASIL DE FUNDACAO OU ASSOCIACAO ESTRANGEIRAS,de_natureza_juridica_ESTABELECIMENTO NO BRASIL DE SOCIEDADE ESTRANGEIRA,de_natureza_juridica_ESTADO OU DISTRITO FEDERAL,de_natureza_juridica_FRENTE PLEBISCITARIA OU REFERENDARIA,de_natureza_juridica_FUNDACAO PRIVADA,de_natureza_juridica_FUNDACAO PUB DE DIREITO PUB EST OU DO DF,de_natureza_juridica_FUNDACAO PUBLICA DE DIREITO PRIVADO FEFERAL,de_natureza_juridica_FUNDACAO PUBLICA DE DIREITO PRIVADO MUNICIPAL,de_natureza_juridica_FUNDACAO PUBLICA DE DIREITO PUB MUNICIPAL,de_natureza_juridica_FUNDACAO PUBLICA DE DIREITO PUBFEDERAL,...,nm_divisao_FABRICACAO DE PRODUTOS QUIMICOS,nm_divisao_FABRICACAO DE PRODUTOS TEXTEIS,nm_divisao_FABRICACAO DE VEICULOS AUTOMOTORES REBOQUES E CARROCERIAS,nm_divisao_IMPRESSAO E REPRODUCAO DE GRAVACOES,nm_divisao_MANUTENCAO REPARACAO E INSTALACAO DE MAQUINAS E EQUIPAMENTOS,nm_divisao_METALURGIA,nm_divisao_OBRAS DE INFRA ESTRUTURA,nm_divisao_ORGANISMOS INTERNACIONAIS E OUTRAS INSTITUICOES EXTRATERRITORIAIS,nm_divisao_OUTRAS ATIVIDADES DE SERVICOS PESSOAIS,nm_divisao_OUTRAS ATIVIDADES PROFISSIONAIS CIENTIFICAS E TECNICAS,nm_divisao_PESCA E AQUICULTURA,nm_divisao_PESQUISA E DESENVOLVIMENTO CIENTIFICO,nm_divisao_PREPARACAO DE COUROS E FABRICACAO DE ARTEFATOS DE COURO ARTIGOS PARA VIAGEM E CALCADOS,nm_divisao_PRODUCAO FLORESTAL,nm_divisao_PUBLICIDADE E PESQUISA DE MERCADO,nm_divisao_REPARACAO E MANUTENCAO DE EQUIPAMENTOS DE INFORMATICA E COMUNICACAO E DE OBJETOS PESSOAIS E DOMESTICOS,nm_divisao_SEGUROS RESSEGUROS PREVIDENCIA COMPLEMENTAR E PLANOS DE SAUDE,nm_divisao_SELECAO AGENCIAMENTO E LOCACAO DE MAO DE OBRA,nm_divisao_SERVICOS DE ARQUITETURA E ENGENHARIA TESTES E ANALISES TECNICAS,nm_divisao_SERVICOS DE ASSISTENCIA SOCIAL SEM ALOJAMENTO,nm_divisao_SERVICOS DE ESCRITORIO DE APOIO ADMINISTRATIVO E OUTROS SERVICOS PRESTADOS PRINCIPALMENTE AS EMPRESAS,nm_divisao_SERVICOS DOMESTICOS,nm_divisao_SERVICOS ESPECIALIZADOS PARA CONSTRUCAO,nm_divisao_SERVICOS PARA EDIFICIOS E ATIVIDADES PAISAGISTICAS,nm_divisao_TELECOMUNICACOES,nm_divisao_TRANSPORTE AEREO,nm_divisao_TRANSPORTE AQUAVIARIO,nm_divisao_TRANSPORTE TERRESTRE,fl_spa_False,fl_spa_True,fl_antt_False,fl_antt_True,fl_veiculo_False,fl_veiculo_True,fl_simples_irregular_False,fl_simples_irregular_True,fl_passivel_iss_False,fl_passivel_iss_True,id,score
110844,True,3.536986,False,False,True,False,False,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,1,9efb64d9016489022eb06941a8a989c3382d21266a60f4...,1.0
427303,True,32.473973,False,False,False,False,False,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0,40320bac6233111923db1e891c4e192f7281fd

In [0]:
df_market[df_market['id'].isin(df_portfolio1['id'].values)].sort_values('score', ascending=False)

,fl_matriz,idade_empresa_anos,fl_me,fl_sa,fl_mei,fl_ltda,fl_st_especial,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,qt_filiais,de_natureza_juridica_ASSOCIACAO PRIVADA,de_natureza_juridica_AUTARQUIA ESTADUAL OU DO DISTRITO FEDERAL,de_natureza_juridica_AUTARQUIA FEDERAL,de_natureza_juridica_AUTARQUIA MUNICIPAL,de_natureza_juridica_CANDIDATO A CARGO POLITICO ELETIVO,de_natureza_juridica_CLUBE FUNDO DE INVESTIMENTO,de_natureza_juridica_COMUNIDADE INDIGENA,de_natureza_juridica_CONDOMINIO EDILICIO,de_natureza_juridica_CONSORCIO DE EMPREGADORES,de_natureza_juridica_CONSORCIO DE SOCIEDADES,de_natureza_juridica_CONSORCIO PUBDE DIREITO PUB ASS PUB,de_natureza_juridica_CONSORCIO PUBLICO DE DIREITO PRIVADO,de_natureza_juridica_COOPERATIVA,de_natureza_juridica_EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITADA DE NATUREZA EMPRESARIA,de_natureza_juridica_EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITADA DE NATUREZA SIMPLES,de_natureza_juridica_EMPRESA INDIVIDUAL IMOBILIARIA,de_natureza_juridica_EMPRESA PUBLICA,de_natureza_juridica_EMPRESARIO INDIVIDUAL,de_natureza_juridica_ENTIDADE DE MEDIACAO E ARBITRAGEM,de_natureza_juridica_ENTIDADE SINDICAL,de_natureza_juridica_ESTABELECIMENTO NO BRASIL DE FUNDACAO OU ASSOCIACAO ESTRANGEIRAS,de_natureza_juridica_ESTABELECIMENTO NO BRASIL DE SOCIEDADE ESTRANGEIRA,de_natureza_juridica_ESTADO OU DISTRITO FEDERAL,de_natureza_juridica_FRENTE PLEBISCITARIA OU REFERENDARIA,de_natureza_juridica_FUNDACAO PRIVADA,de_natureza_juridica_FUNDACAO PUB DE DIREITO PUB EST OU DO DF,de_natureza_juridica_FUNDACAO PUBLICA DE DIREITO PRIVADO FEFERAL,de_natureza_juridica_FUNDACAO PUBLICA DE DIREITO PRIVADO MUNICIPAL,de_natureza_juridica_FUNDACAO PUBLICA DE DIREITO PUB MUNICIPAL,de_natureza_juridica_FUNDACAO PUBLICA DE DIREITO PUBFEDERAL,...,nm_divisao_FABRICACAO DE PRODUTOS QUIMICOS,nm_divisao_FABRICACAO DE PRODUTOS TEXTEIS,nm_divisao_FABRICACAO DE VEICULOS AUTOMOTORES REBOQUES E CARROCERIAS,nm_divisao_IMPRESSAO E REPRODUCAO DE GRAVACOES,nm_divisao_MANUTENCAO REPARACAO E INSTALACAO DE MAQUINAS E EQUIPAMENTOS,nm_divisao_METALURGIA,nm_divisao_OBRAS DE INFRA ESTRUTURA,nm_divisao_ORGANISMOS INTERNACIONAIS E OUTRAS INSTITUICOES EXTRATERRITORIAIS,nm_divisao_OUTRAS ATIVIDADES DE SERVICOS PESSOAIS,nm_divisao_OUTRAS ATIVIDADES PROFISSIONAIS CIENTIFICAS E TECNICAS,nm_divisao_PESCA E AQUICULTURA,nm_divisao_PESQUISA E DESENVOLVIMENTO CIENTIFICO,nm_divisao_PREPARACAO DE COUROS E FABRICACAO DE ARTEFATOS DE COURO ARTIGOS PARA VIAGEM E CALCADOS,nm_divisao_PRODUCAO FLORESTAL,nm_divisao_PUBLICIDADE E PESQUISA DE MERCADO,nm_divisao_REPARACAO E MANUTENCAO DE EQUIPAMENTOS DE INFORMATICA E COMUNICACAO E DE OBJETOS PESSOAIS E DOMESTICOS,nm_divisao_SEGUROS RESSEGUROS PREVIDENCIA COMPLEMENTAR E PLANOS DE SAUDE,nm_divisao_SELECAO AGENCIAMENTO E LOCACAO DE MAO DE OBRA,nm_divisao_SERVICOS DE ARQUITETURA E ENGENHARIA TESTES E ANALISES TECNICAS,nm_divisao_SERVICOS DE ASSISTENCIA SOCIAL SEM ALOJAMENTO,nm_divisao_SERVICOS DE ESCRITORIO DE APOIO ADMINISTRATIVO E OUTROS SERVICOS PRESTADOS PRINCIPALMENTE AS EMPRESAS,nm_divisao_SERVICOS DOMESTICOS,nm_divisao_SERVICOS ESPECIALIZADOS PARA CONSTRUCAO,nm_divisao_SERVICOS PARA EDIFICIOS E ATIVIDADES PAISAGISTICAS,nm_divisao_TELECOMUNICACOES,nm_divisao_TRANSPORTE AEREO,nm_divisao_TRANSPORTE AQUAVIARIO,nm_divisao_TRANSPORTE TERRESTRE,fl_spa_False,fl_spa_True,fl_antt_False,fl_antt_True,fl_veiculo_False,fl_veiculo_True,fl_simples_irregular_False,fl_simples_irregular_True,fl_passivel_iss_False,fl_passivel_iss_True,id,score
110844,True,3.536986,False,False,True,False,False,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,1,9efb64d9016489022eb06941a8a989c3382d21266a60f4...,1.000000
427303,True,32.473973,False,False,False,False,False,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0,40320bac6233111923db1e891c4e192f7